<a href="https://colab.research.google.com/github/farida752/8-Puzzle-game/blob/main/AI_Searching_Algorithms_8_puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("input")
my_hexdata = input().replace(",", "")
zero_index_i =  my_hexdata.find('0')//3
zero_index_j =  my_hexdata.find('0')-(zero_index_i * 3)
print(str(zero_index_i) + str( zero_index_j))

scale = 16 ## equals to hexadecimal

num_of_bits = 36

initail_state = bin(int(my_hexdata, scale))[2:].zfill(num_of_bits)


input
1,2,3,4,5,0,6,7,8
12


In [ ]:
function constructNextState (state , action , zero_index_i , zero_index_j):
  if action == 'up':
    # temp 
